<a href="https://colab.research.google.com/github/umaremlyn/LeenCo-Site/blob/main/Xamani_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers accelerate einops bitsandbytes sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 17.3 MB/s eta 0:00:00


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from google.colab import userdata

model_id = "tiiuae/falcon-7b-instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load model in 4-bit quantized mode (saves GPU memory)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True
)

# Build pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

# Test
prompt = "Explain the benefits of using AI for small businesses in Africa."
outputs = pipe(prompt, max_new_tokens=200, temperature=0.7, do_sample=True)
print(outputs[0]['generated_text'])

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Explain the benefits of using AI for small businesses in Africa.
AI benefits small African businesses in numerous ways. Firstly, AI can help reduce labor costs by automating tedious, repetitive tasks, thus freeing human resources for more strategic work. Secondly, AI can be used to analyze and make sense of large amounts of data, enabling small businesses to make better decisions and predictions. Thirdly, AI can be used to develop new products and services, giving small businesses the opportunity to stay ahead of their competitors. Finally, AI can be used to increase customer satisfaction and loyalty by providing fast, personalized, and efficient services.


In [6]:
!pip install -q transformers accelerate bitsandbytes

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "tiiuae/falcon-7b-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=False
)

# 🔹 Prompt Refiner Function
def refine_prompt(user_input, max_new_tokens=120):
    system_instruction = (
        "You are a professional AI prompt engineer. "
        "Rewrite the following user input into a clear, structured AI prompt. "
        "Only return the refined prompt, nothing else.\n\n"
        f"User input: {user_input}\n\nRefined prompt:"
    )
    inputs = tokenizer(system_instruction, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 🔹 Example usage
user_raw = "Help me with a CV"  # messy input
refined = refine_prompt(user_raw)
print("🔹 User Input:", user_raw)
print("✅ Refined Prompt:", refined)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


🔹 User Input: Help me with a CV
✅ Refined Prompt: You are a professional AI prompt engineer. Rewrite the following user input into a clear, structured AI prompt. Only return the refined prompt, nothing else.

User input: Help me with a CV

Refined prompt: Can you provide assistance with creating a CV tailored to my skills and experience?
